In [2]:
library(rtracklayer)
library(ComplexHeatmap)
library(ChIPseeker)
library(TxDb.Hsapiens.UCSC.hg19.knownGene)
txdb <- TxDb.Hsapiens.UCSC.hg19.knownGene
library(org.Hs.eg.db)
library(clusterProfiler)
library(DESeq2)
library(edgeR)

Loading required package: grid

ComplexHeatmap version 2.14.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite either one:
- Gu, Z. Complex Heatmap Visualization. iMeta 2022.
- Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
    genomic data. Bioinformatics 2016.


The new InteractiveComplexHeatmap package can directly export static 
complex heatmaps into an interactive Shiny app with zero effort. Have a try!

This message can be suppressed by:
  suppressPackageStartupMessages(library(ComplexHeatmap))




Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘matrixStats’


The following objects are masked from ‘package:Biobase’:

    anyMissing, rowMedians


The following ob

In [20]:
#sample_infor = read.table(file= "/Users/kumarr9/Downloads/ATAC_data/ATAC_final_files/df4.tsv", header = T, sep = "\t")
atac_df = read.table(file= "/data/SCLCgenomics/rajesh/ATAC_scripts/ATAC_analysis/ATAC_batch_1_2_3_quantified_peaks/homer_analysis/differential_peak_results/DGE_parth_gene_annotation/rank3.pdx.cluster3.DEseq2.tsv", header = T, sep = "\t", row.names = 1, check.names = F)
#colnames(atac_df) = gsub(pattern = ".sorted.dedup.bam", replacement = "", colnames(atac_df)) ####removing pattern from the colnames


In [21]:
####  getting gene length #################
gene_lengths = data.frame(GeneID = rownames(atac_df),
                          Length = width(GRanges(rownames(atac_df))))

rownames(gene_lengths) = gene_lengths$GeneID

In [22]:
peakAnno <- annotatePeak(GRanges(rownames(atac_df)), tssRegion=c(-3000, 3000),
                         TxDb=txdb, annoDb="org.Hs.eg.db")

peak_df <- as.data.frame(peakAnno@anno)
peak_df$coordinate = paste0(peak_df$seqnames, ":", peak_df$start, "-", peak_df$end)
peak_df = peak_df[!duplicated(peak_df$coordinate),]
rownames(peak_df) = peak_df$coordinate

>> preparing features information...		 2024-03-08 14:28:43 
>> identifying nearest features...		 2024-03-08 14:28:43 
>> calculating distance from peak to TSS...	 2024-03-08 14:28:49 
>> assigning genomic annotation...		 2024-03-08 14:28:49 
>> adding gene annotation...			 2024-03-08 14:28:56 


'select()' returned 1:many mapping between keys and columns



>> assigning chromosome lengths			 2024-03-08 14:28:56 
>> done...					 2024-03-08 14:28:56 


In [23]:
peak_df_test = peak_df
atac_df_test <- atac_df
atac_df_test$Gene <- peak_df_test$SYMBOL
atac_df_test$annotation <- peak_df_test$annotation

In [24]:
head(atac_df_test)

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Gene,annotation
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
chr1:9997-10674,153.062790,-0.62270224,0.1571248,-3.9631048,7.398127e-05,0.000690813,DDX11L1,Promoter (1-2kb)
chr1:11194-11315,23.205062,-0.26906558,0.2107789,-1.2765300,2.017682e-01,0.316372018,DDX11L1,Promoter (<=1kb)
chr1:12764-15682,13.552618,0.64493331,0.1785535,3.6119901,3.038560e-04,0.002013147,DDX11L1,Promoter (<=1kb)
chr1:16814-16958,14.091000,-0.02871356,0.1480029,-0.1940068,8.461706e-01,0.896341280,WASH7P,Promoter (<=1kb)
chr1:17185-17380,12.078361,-0.22724870,0.1436957,-1.5814574,1.137735e-01,0.203905728,WASH7P,Promoter (<=1kb)
chr1:18209-18770,7.121041,0.16081936,0.2296818,0.7001834,4.838128e-01,0.606414973,WASH7P,Promoter (<=1kb)


In [25]:
write.table(atac_df_test, "/data/SCLCgenomics/rajesh/ATAC_scripts/ATAC_analysis/ATAC_batch_1_2_3_quantified_peaks/homer_analysis/differential_peak_results/DGE_parth_gene_annotation/rank3.pdx.cluster3.DEseq2_gene_mapped.tsv", sep="\t", row.names=TRUE)
